In [1]:
import folium
from geopy.distance import geodesic, great_circle
import geopandas as gpd
import pandas as pd
import pyproj
from folium import plugins
import warnings
warnings.simplefilter("ignore")

In [2]:
# load geo json
up = gpd.read_file(r"F:/DSLR/Folium/Geojson/Upazilla_updated.geojson")

# From Zero

In [3]:
# filter the required upazilla according to requirement
upzs = ['Biral', 'Birganj', 'Kaharole', 'Khansama']
temp_geo = up[up.ADM3_EN.isin(upzs)].reset_index(drop=True)

In [5]:
def style_function(fill_color,fill_opacity):
    return lambda feature: {
        'color': "green",
        'weight': 2,
        'fillColor': fill_color,
        'fillOpacity': fill_opacity
    }

def get_centre_point(lat, lon):
    centre = folium.Circle(location=[lat, lon],
                          radius= 200,
                          color='blue',
                          fill_color = 'blue',
                          fillOpacity = 1,
                          tooltip = str(lat) + "<br>" + str(lon),
                          stroke = True)
    return centre

def get_centre_point_2(lat, lon, tt):
    centre = folium.Circle(location=[lat, lon],
                          radius= 200,
                          color='blue',
                          fill_color = 'blue',
                          fillOpacity = 1,
                          tooltip = tt,
                          stroke = True)
    return centre

def plot_poly(poly, lat, lon, ind, rad, m, toggle):
    new_lat, new_lon, new_poly = get_mirror(poly, lat, lon, ind, rad)
    if toggle == True:
        return new_poly, new_lat, new_lon
    poly = folium.Polygon(locations=new_poly,
                          tooltip = str(new_lat) + '<br>' + str(new_lon),
                           fill_color='orange', color = 'orange', fillOpacity = 0.001).add_to(m)
    c = get_centre_point(new_lat, new_lon)
    c.add_to(m)
    return m

def get_mirror(poly, cen_lat, cen_lon, ind, rad):
    latA, lonA = cen_lat, cen_lon
    latB, lonB = poly[ind]
    latC, lonC = poly[ind+1]
    latD, lonD = latB + (latC - latA), lonB + (lonC - lonA)
    return latD, lonD, create_polygon((latD, lonD), rad, 6)

def create_polygon(centre, radius_km, num_points=6):
    polygon = []

    for i in range(num_points):
        angle = 360.0 * i / num_points
        point = geodesic(kilometers=radius_km).destination(centre, angle)
        polygon.append((point.latitude, point.longitude))
    polygon.append(polygon[0])
    return polygon

In [9]:
temp_geo

,Shape_Leng,Shape_Area,ADM3_EN,ADM3_PCODE,ADM3_REF,ADM3ALT1EN,ADM3ALT2EN,ADM2_EN,ADM2_PCODE,ADM1_EN,ADM1_PCODE,ADM0_EN,ADM0_PCODE,date,validOn,validTo,geometry
0,1.093615,0.031585,Biral,BD552717,None,None,None,Dinajpur,BD5527,Rangpur,BD55,Bangladesh,BD,2015-01-01,2020-11-13,None,"MULTIPOLYGON (((88.52806 25.76603, 88.52820 25..."
1,1.305429,0.037414,Birganj,BD552712,None,None,None,Dinajpur,BD5527,Rangpur,BD55,Bangladesh,BD,2015-01-01,2020-11-13,None,"MULTIPOLYGON (((88.62406 26.05703, 88.62421 26..."
2,0.937103,0.018512,Kaharole,BD552756,None,None,None,Dinajpur,BD5527,Rangpur,BD55,Bangladesh,BD,2015-01-01,2020-11-13,None,"MULTIPOLYGON (((88.55319 25.88093, 88.55335 25..."
3,0.859187,0.016239,Khansama,BD552760,None,None,None,Dinajpur,BD5527,Rangpur,BD55,Bangladesh,BD,2015-01-01,2020-11-13,None,"MULTIPOLYGON (((88.73488 26.01469, 88.73490 26..."


In [6]:
rad = 3.5
inc_y = 0
inc_x = 0
centre = temp_geo.geometry[0].centroid
cen = [centre.y, centre.x]

poly = create_polygon((centre.y + inc_y, centre.x + inc_x), rad, 6)


mapObj_dist = folium.Map(location=cen, zoom_start=10, attr='Google', 
                         tiles='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}')
folium.GeoJson(
            data=temp_geo,
            name=temp_geo.ADM2_EN[0],
            tooltip = temp_geo.ADM2_EN[0],
            style_function= style_function('red', 0.7),
            highlight_function= lambda feat: {'fillColor': 'red', 'fillOpacity': 0.3, 'weight': 2, 'color': 'green'}
                ).add_to(mapObj_dist)

c = get_centre_point(centre.y + inc_y, centre.x + inc_x)
c.add_to(mapObj_dist)

for i in range(6):
    mapObj_dist = plot_poly(poly, centre.y + inc_y, centre.x + inc_x, i, rad, mapObj_dist, False)
    
mapObj_dist